<a href="https://colab.research.google.com/github/seyirex/Hamoye_Notebook_repo/blob/main/Hamoye_internship_notebook_stage_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Downloading dataset from gdrive
! gdown --id 1gAhm422HgvwuY34Drq4u59ttpIQC66Om

Downloading...
From: https://drive.google.com/uc?id=1gAhm422HgvwuY34Drq4u59ttpIQC66Om
To: /content/energydata_complete.csv
100% 12.0M/12.0M [00:00<00:00, 17.0MB/s]


In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

In [2]:
train= pd.read_csv('energydata_complete.csv')

In [3]:
# check data shapes
train.shape

(19735, 29)

In [4]:
# Preview train
train.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

In [12]:
train.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

There are no null values in the data
As we are not trying to analyze the problem as Time Series rather regress on "Appliance" column therefore we are dropping the date column

In [5]:
train.drop(['date'], 1, inplace = True)
print(train.columns)

Index(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9',
       'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')


In [4]:
## Divide the columns based on type for clear column management 
temp_col = ["T1","T2","T3","T4","T5","T6","T7","T8","T9"]
hum_col = ["RH_1","RH_2","RH_3","RH_4","RH_5","RH_6","RH_7","RH_8","RH_9"]
weather_col = ["T_out", "Tdewpoint","RH_out","Press_mm_hg", "Windspeed","Visibility"] 
randoms_col = ["rv1", "rv2"]
target_col = ["Appliances"]

In [5]:
# Seperate dependent and independent variables 
feature_vars = train[temp_col + hum_col + weather_col +randoms_col ]
target_vars = train[target_col]

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(feature_vars), columns=feature_vars.columns)

In [6]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(train), columns=train.columns)
X = normalised_df.drop(columns=['lights'])
y = normalised_df["Appliances"]


In [7]:
#Now, we split our dataset into the training and testing dataset. Recall that we
#had earlier segmented the features and target variables.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=0.3, random_state=42)

In [14]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train[['T2']], X_train.T6)
predicted_values = linear_model.predict(X_test[['T2']])

In [9]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
linear_model_1 = LinearRegression()
linear_model_1.fit(X_train, X_train)
predicted_values = linear_model_1.predict(X_test)

In [18]:
#R-Squared
from sklearn.metrics import r2_score
r2_score = r2_score(X_test.T6, predicted_values)
round(r2_score, 2) 

0.64

In [19]:
#Root Mean Square Error (MSE) 
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2) 

0.33

In [16]:
#Residual Sum of Squares (RSS) 
import numpy as np
rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2) #

776.73

In [20]:
#Root Mean Square Error (RMSE) 
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 2) 

0.36

In [21]:
#R-Squared
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2) 

-13.58

In [31]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)
predicted_values1 = lasso_reg.predict(X_test)

In [35]:
#Root Mean Square Error (RMSE) 
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values1))
round(rmse, 3) 

0.01

In [36]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)
predicted_values2 =ridge_reg.predict(X_test)

In [37]:
#Root Mean Square Error (RMSE) 
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values2))
round(rmse, 3) 

0.0

In [25]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df


In [27]:
# linear_model_weights = get_weights_df(linear_model_1, X_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, X_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')

final_weights = pd.merge(lasso_weights_df, ridge_weights_df, on='Features')
# final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')

In [28]:
final_weights

,Features,Lasso_weight,Ridge_Weight
0,T7,0.000000,0.000037
1,Tdewpoint,0.000000,0.000302
2,Visibility,0.000000,0.000044
3,Windspeed,0.000000,0.000110
4,RH_out,-0.000000,-0.000199
5,Press_mm_hg,-0.000000,0.000024
6,T_out,0.000000,-0.000952
7,RH_9,-0.000000,-0.000150
8,T9,0.000000,-0.000686
9,RH_8,-0.000000,-0.000570
